## Load Library

In [3]:
import math
import time

import numpy as np
import pandas as pd
import networkx as nx

import matplotlib.pyplot as plt
%matplotlib inline

## Read File

In [4]:
inFile = open("tsp.txt", "r")

## Convert File to Pandas

In [5]:
arr_list = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    arr_list.append(line.split(" "))
    
df = pd.DataFrame(arr_list, columns = ["Name", "X", "Y"])

# Convert Value From String to int
df["X"] = df["X"].astype(int)
df["Y"] = df["Y"].astype(int)

In [6]:
df

,Name,X,Y
0,Depot,299,153
1,Point1,359,164
2,Parcel2,408,161
3,Point3,378,200
4,Parcel4,452,211
5,Point5,389,250
6,Parcel6,278,225
7,Point7,326,220
8,Parcel8,297,186
9,Parcel9,212,123


In [7]:
def distance(p1, p2):
    return math.sqrt((p1.x - p2.x) ** 2 + (p1.y - p2.y) ** 2)

In [8]:
class Point:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y
        
    def __str__(self):
        return f"Name: {self.name}, X Pos: {self.x}, Y Pos: {self.y}"

## Create Point Array

In [9]:
pt_arr = []

inFile.seek(0)
for line in inFile.readlines():
    line = line[:-1]
    tmp_arr = line.split(" ")
    
    pt_name = tmp_arr[0]
    x_pos = int(tmp_arr[1])
    y_pos = int(tmp_arr[2])
    
    pt = Point(pt_name, x_pos, y_pos)
    
    pt_arr.append(pt)
    
list(map(lambda x : x.name, pt_arr))

['Depot',
 'Point1',
 'Parcel2',
 'Point3',
 'Parcel4',
 'Point5',
 'Parcel6',
 'Point7',
 'Parcel8',
 'Parcel9',
 'Parcel10',
 'Point11',
 'Point12',
 'Parcel13',
 'Parcel14',
 'Parcel15']

Create Graph\
Depot => Parcel\
Parcel => Point\
Point => Parcel

In [10]:
def create_point(ind):
    row = df.iloc[ind, :]
    return Point(row.Name, row.X, row.Y)

In [11]:
def add_edge_to_graph(G, e1, e2, w):
    G.add_edge(e1, e2, weight=w)

## Pairwise Table

In [12]:
arr_list = []

name_arr = list(map(lambda x : x.name, pt_arr))

name_dict = {name:str(ind) for (ind, name) in enumerate(name_arr)}

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    if "Depot" in name:
        for pt2 in pt_arr:
            if "Parcel" in pt2.name:
                arr_list.append([name_dict[name], name_dict[pt2.name], str(distance(pt, pt2))])
    elif "Parcel" in name:
        for pt2 in pt_arr:
            if "Point" in pt2.name:
                arr_list.append([name_dict[name], name_dict[pt2.name], str(distance(pt, pt2))])
    elif "Point" in name:
        for pt2 in pt_arr:
            if "Parcel" in pt2.name:
                arr_list.append([name_dict[name], name_dict[pt2.name], str(distance(pt, pt2))])
    
for f in arr_list:
    print(" ".join(f))

0 2 109.29318368498559
0 4 163.62457028209425
0 6 75.0
0 8 33.06055050963308
0 9 92.02716990106781
0 10 53.16013544000805
0 13 174.42763542512407
0 14 206.60348496576722
0 15 144.01388821915754
1 2 49.09175083453431
1 4 104.20172743289815
1 6 101.40019723846694
1 8 65.7875368135941
1 9 152.6106156202772
1 10 76.60939890117922
1 13 145.38225476308997
1 14 154.20765220960988
1 15 190.2761151589973
2 1 49.09175083453431
2 3 49.20365840057018
2 5 91.00549433962765
2 7 101.01980003939822
2 11 45.31004303683677
2 12 197.3068675946177
3 2 49.20365840057018
3 4 74.81310045707235
3 6 103.07764064044152
3 8 82.20097323024831
3 9 182.98907071188705
3 10 117.04699910719626
3 13 108.00462953040486
3 14 115.52056094046635
3 15 197.3068675946177
4 1 104.20172743289815
4 3 74.81310045707235
4 5 74.09453421137081
4 7 126.3210196285638
4 11 95.88013350011565
4 12 198.600100704909
5 2 91.00549433962765
5 4 74.09453421137081
5 6 113.78049041905207
5 8 112.0714058089752
5 9 217.84857125994654
5 10 165.9186

## Make Adjacency Matrix

In [13]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    if "Depot" in name:
        for pt2 in pt_arr:
            dist_val = distance(pt, pt2) if "Parcel" in pt2.name else 0
            tmp_arr.append(round(dist_val, 3))
    elif "Parcel" in name:
        for pt2 in pt_arr:
            dist_val = distance(pt, pt2) if "Point" in pt2.name or "Parcel" in pt2.name else 0
            tmp_arr.append(round(dist_val, 3))
    elif "Point" in name:
        for pt2 in pt_arr:
            dist_val = distance(pt, pt2) if "Parcel" in pt2.name or "Depot" in pt2.name else 0
            tmp_arr.append(round(dist_val, 3))
    
    adj_mat.append(tmp_arr)
    
adj_df = pd.DataFrame(adj_mat, columns = df["Name"].tolist(), index = df["Name"].tolist())

# Convert adj_df to Float Type
adj_df = adj_df.astype(float)

adj_df

,Depot,Point1,Parcel2,Point3,Parcel4,Point5,Parcel6,Point7,Parcel8,Parcel9,Parcel10,Point11,Point12,Parcel13,Parcel14,Parcel15
Depot,0.000,0.000,109.293,0.000,163.625,0.000,75.000,0.000,33.061,92.027,53.160,0.000,0.000,174.428,206.603,144.014
Point1,61.000,0.000,49.092,0.000,104.202,0.000,101.400,0.000,65.788,152.611,76.609,0.000,0.000,145.382,154.208,190.276
Parcel2,0.000,49.092,0.000,49.204,66.603,91.005,144.900,101.020,113.780,199.650,110.982,45.310,197.307,149.833,128.351,236.916
Point3,91.924,0.000,49.204,0.000,74.813,0.000,103.078,0.000,82.201,182.989,117.047,0.000,0.000,108.005,115.521,197.307
Parcel4,0.000,104.202,66.603,74.813,0.000,74.095,174.562,126.321,157.003,255.625,175.855,95.880,198.600,121.400,66.287,268.680
Point5,132.322,0.000,91.005,0.000,74.095,0.000,113.780,0.000,112.071,217.849,165.919,0.000,0.000,58.856,80.324,204.198
Parcel6,0.000,101.400,144.900,103.078,174.562,113.780,0.000,48.260,43.382,121.491,128.160,181.232,87.052,130.729,193.830,94.366
Point7,72.236,0.000,101.020,0.000,126.321,0.000,48.260,0.000,44.688,149.683,118.609,0.000,0.000,102.728,149.857,142.555
Parcel8,0.000,65.788,113.780,82.201,157.003,112.071,43.382,44.688,0.000,105.802,85.703,144.475,127.012,146.997,190.589,124.776
Parcel9,0.000,152.611,199.650,182.989,255.625,217.849,121.491,149.683,105.802,0.000,104.139,213.038,201.201,249.227,295.665,121.050


In [14]:
class Path:
    def __init__(self, path, cost):
        self.path = path
        self.cost = cost
        
    def __lt__(self, other):
        return self.cost <= other.cost
        
    def __str__(self):
        return f"Shortest Path:{self.path}\nMinimum Cost: {self.cost}"

## DFS (With Backtracking)

In [15]:
def dfs_adv2(adj_mat, visited, path, cost, condition_arr, cur_node):
    visited[cur_node] = True
    path.append(cur_node)
    
    min_path = Path([], float('inf'))
    
    if len(path) > 1:
        cost += adj_mat[path[-2]][path[-1]]
        
    tmp_arr = [visited[ind] for ind in condition_arr]
    
    if min(tmp_arr) == 1:
        cost += adj_mat[cur_node][0]
        min_path = Path(path.copy() + [0], cost)
    else:
        for i in range(len(adj_mat)):
            if visited[i] == False and adj_mat[cur_node][i] != 0:
                min_path2 = dfs_adv2(adj_mat, visited, path, cost, condition_arr, i)
                min_path = min(min_path, min_path2)
            
    path.pop()
    visited[cur_node] = False
    
    return min_path
    
def dfs_adv(adj_mat, name_arr, start_ind):
    visited = [False] * len(adj_mat)
    
    path = []
    
    cost = 0
    
    condition = [ind for (ind, val) in enumerate(name_arr) if "Point" in val]
    
    return dfs_adv2(adj_mat, visited, path, cost, condition, start_ind)

In [16]:
start = time.process_time()
print(dfs_adv(adj_mat, name_arr, 0))
print(f"Time Taken: {time.process_time() - start}")

KeyboardInterrupt: 

## Make Adjacency List

In [15]:
adj_list = {}

tmp_nm_dict = {name:ind for (ind, name) in enumerate(name_arr)}

for pt in pt_arr:
    
    name = pt.name
    
    tmp_dict = {}
        
    if "Depot" in name:
        for pt2 in pt_arr:
            if "Parcel" in pt2.name:
                dist_val = distance(pt, pt2)
                tmp_dict[tmp_nm_dict[pt2.name]] = round(dist_val, 3)
    elif "Parcel" in name:
        for pt2 in pt_arr:
            if "Point" in pt2.name:
                dist_val = distance(pt, pt2)
                tmp_dict[tmp_nm_dict[pt2.name]] = round(dist_val, 3)
    elif "Point" in name:
        for pt2 in pt_arr:
            if "Parcel" in pt2.name:
                dist_val = distance(pt, pt2)
                tmp_dict[tmp_nm_dict[pt2.name]] = round(dist_val, 3)

    adj_list[tmp_nm_dict[pt.name]] = tmp_dict

In [16]:
adj_list

{0: {1: 135.532, 3: 251.336, 5: 334.646},
 1: {2: 226.927, 4: 359.221},
 2: {1: 226.927, 3: 157.153, 5: 164.842},
 3: {2: 157.153, 4: 168.158},
 4: {1: 359.221, 3: 168.158, 5: 129.557},
 5: {2: 164.842, 4: 129.557}}

## Adjacency Matrix

In [17]:
adj_mat = []

name_arr = list(map(lambda x : x.name, pt_arr))

dist_val = 0

for pt in pt_arr:
    
    name = pt.name
    
    tmp_arr = []
    
    for pt2 in pt_arr:
        dist_val = round(distance(pt, pt2))
        if pt.name == pt2.name or ("Depot" in pt.name and "Point" in pt2.name) or ("Parcel" in pt.name and "Depot" in pt2.name) or ("Point" in pt.name and "Point" in pt2.name) or ("Parcel" in pt.name and "Parcel" in pt2.name):
            dist_val = float('inf')
        tmp_arr.append(dist_val)
    
    adj_mat.append(tmp_arr)
    
adj_df = pd.DataFrame(adj_mat, columns = df["Name"].tolist(), index = df["Name"].tolist())

# Convert adj_df to Float Type
adj_df = adj_df.astype(float)

adj_df

,Depot,Parcel1,Point2,Parcel3,Point4,Parcel5
Depot,inf,136.0,inf,251.0,inf,335.0
Parcel1,inf,inf,227.0,inf,359.0,inf
Point2,174.0,227.0,inf,157.0,inf,165.0
Parcel3,inf,inf,157.0,inf,168.0,inf
Point4,355.0,359.0,inf,168.0,inf,130.0
Parcel5,inf,inf,165.0,inf,130.0,inf
